In [1]:

import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout,Concatenate,Input, LSTM,GRU, Flatten, Conv1D, MaxPooling1D,Bidirectional
import pandas as pd 
from pandas import read_csv
import matplotlib.pyplot as pyplot
import numpy as np 
from numpy import concatenate
from tqdm import tqdm
import math
import datetime
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
num_folds = 10


In [3]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 750 Ti, pci bus id: 0000:01:00.0, compute capability: 5.0



In [4]:
%matplotlib inline

In [5]:
seed_value=21

In [6]:
tf.random.set_seed(seed_value)

In [7]:
train=pd.read_csv("./data/processed_train.csv")
test=pd.read_csv("./data/processed_test.csv")

In [8]:
train.drop(15538,inplace=True)

In [9]:
targets = train["target"]
train.drop(["target"],axis=1,inplace=True)
result_array = train.values
result_test_array = test.values
result_array = np.nan_to_num(result_array)
result_test_array = np.nan_to_num(result_test_array)

In [10]:
features = np.array_split(train,7,axis=1)

In [11]:
for index,feat in enumerate(features,start=0):
    features[index] = features[index].values
    features[index] = features[index].reshape(15538,121,1)

In [12]:
x_train = np.reshape(result_array,(15538,121,7))
x_test = np.reshape(result_test_array,(5035,121,7))

In [13]:
x_train_one = np.reshape(result_array,(15538,847,1))
x_test_one = np.reshape(result_test_array,(5035,847,1))

In [14]:
y_train = targets.values

In [15]:
y_train_cnn = targets.values

## k-cross validation GRU

In [44]:
GRUloss = []
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1

In [45]:
for train, test in kfold.split(x_train, y_train):
    print("fold",fold_no)
    KgruModel = Sequential()
    KgruModel.add(GRU(10,input_shape=(121,7), activation ="relu", return_sequences=False))
    KgruModel.add(Dropout(0.3,seed=seed_value))
    KgruModel.add(Dense(255, activation="relu"))
    KgruModel.add(Dense(1))
    KgruModel.compile(loss="mae",optimizer="adam")
    kFoldModelScores = KgruModel.fit(x_train[train],y_train[train],epochs=10,verbose=1)
    
    scores = KgruModel.evaluate(x_train[test], y_train[test], verbose=1)
    print(scores)
    
    GRUloss.append(scores)
    fold_no = fold_no + 1
    
    


fold 1
Train on 13984 samples
Epoch 1/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 27.4444
Epoch 2/10
13984/13984 [==============================] - 37s 3ms/sample - loss: 26.8646
Epoch 3/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 26.8082
Epoch 4/10
13984/13984 [==============================] - 37s 3ms/sample - loss: 26.6697
Epoch 5/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 26.6745
Epoch 6/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 26.7043
Epoch 7/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 26.7037
Epoch 8/10
13984/13984 [==============================] - 38s 3ms/sample - loss: 26.6716
Epoch 9/10
13984/13984 [==============================] - 37s 3ms/sample - loss: 26.6668
Epoch 10/10
1554/1554 [==============================] - 1s 720us/sample - loss: 28.6886
28.688612379469312
fold 2
Train on 13984 samples
Epoch 1/10
13984/13984 [=======

13985/13985 [==============================] - 46s 3ms/sample - loss: 29.2147
Epoch 2/10
13985/13985 [==============================] - 45s 3ms/sample - loss: 27.5397
Epoch 3/10
13985/13985 [==============================] - 45s 3ms/sample - loss: 26.9848
Epoch 4/10
13985/13985 [==============================] - 46s 3ms/sample - loss: 26.9323
Epoch 5/10
13985/13985 [==============================] - 45s 3ms/sample - loss: 26.8370
Epoch 6/10
13985/13985 [==============================] - 44s 3ms/sample - loss: 26.6900
Epoch 7/10
13985/13985 [==============================] - 44s 3ms/sample - loss: 26.6651
Epoch 8/10
13985/13985 [==============================] - 45s 3ms/sample - loss: 26.8736
Epoch 9/10
13985/13985 [==============================] - 44s 3ms/sample - loss: 26.6620
Epoch 10/10
1553/1553 [==============================] - 1s 730us/sample - loss: 30.5018
30.501773261901107
fold 10
Train on 13985 samples
Epoch 1/10
13985/13985 [==============================] - 45s 3ms/sampl

In [46]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(GRUloss), np.std(GRUloss)))

29.90% (+/- 2.06%)


## k-cross validation LSTM

In [41]:
LSTMloss = []
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1

In [42]:
for train, test in kfold.split(x_train, y_train):
    print("fold",fold_no)
    kFoldModel = Sequential()
    kFoldModel.add(LSTM(12, return_sequences=False,activation="relu", input_shape=(121,7)))
    kFoldModel.add(Dropout(0.2))
    kFoldModel.add(Dense(50, activation="relu"))
    kFoldModel.add(Dense(1))
    kFoldModel.compile(loss="mae",optimizer="adam")
    kFoldModelScores = kFoldModel.fit(x_train[train],y_train[train],epochs=10,verbose=1)
    
    scores = kFoldModel.evaluate(x_train[test], y_train[test], verbose=1)
    print(scores)
    
    LSTMloss.append(scores)
    fold_no = fold_no + 1
    
                                              
    

fold 1
Train on 13984 samples
Epoch 1/10
13984/13984 [==============================] - 34s 2ms/sample - loss: 35.1676
Epoch 2/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 30.1253
Epoch 3/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 29.9282
Epoch 4/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 29.4605
Epoch 5/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 29.2300
Epoch 6/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 28.8989
Epoch 7/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 28.8505
Epoch 8/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 28.7027
Epoch 9/10
13984/13984 [==============================] - 33s 2ms/sample - loss: 28.7578
Epoch 10/10
1554/1554 [==============================] - 1s 591us/sample - loss: 25.4562
25.45621188052066
fold 2
Train on 13984 samples
Epoch 1/10
13984/13984 [========

13985/13985 [==============================] - 38s 3ms/sample - loss: 35.4108
Epoch 2/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 39.4555
Epoch 3/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 37.6935
Epoch 4/10
13985/13985 [==============================] - 37s 3ms/sample - loss: 34.7887
Epoch 5/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 33.3608
Epoch 6/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 32.1482
Epoch 7/10
13985/13985 [==============================] - 37s 3ms/sample - loss: 30.7179
Epoch 8/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 28.9790
Epoch 9/10
13985/13985 [==============================] - 36s 3ms/sample - loss: 27.7987
Epoch 10/10
1553/1553 [==============================] - 1s 591us/sample - loss: 27.0413
27.041331569379327
fold 10
Train on 13985 samples
Epoch 1/10
13985/13985 [==============================] - 37s 3ms/sampl

In [43]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(LSTMloss), np.std(LSTMloss)))

393.93% (+/- 833.50%)


## LSTM(847,1)

In [ ]:
lstmModel = Sequential()
lstmModel.add(LSTM(10,input_shape=(847,1), activation ="relu", return_sequences=False))
lstmModel.add(Dropout(0.3,seed=seed_value))
lstmModel.add(Dense(50, activation="relu"))
lstmModel.add(Dense(1))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
lstmModel.compile(loss="mae",optimizer="adam",metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
lstmModel.summary()

In [ ]:
lstmOneHistory = lstmModel.fit(x_train_one,y_train,validation_split=0.3, epochs=50, shuffle=True)

In [ ]:
lstmPredicitions = lstmModel.predict(x_train_one)

In [ ]:
fig = pyplot.figure()
fig.suptitle('DENSE CNN PREDICTIONS', fontsize=14)


pyplot.plot(y_train,label="actual targets", color="blue")
pyplot.plot(lstmPredicitions,label="predicted targets", color="red")


pyplot.xlabel('sample')
pyplot.ylabel('target')

pyplot.legend()
pyplot.show()

## LSTM (121,7)

In [ ]:
lstmModel2 = Sequential()
# lstmModel2.add(Bidirectional(LSTM(12,input_shape=(121,7), activation ="relu",return_sequences=True)))
lstmModel2.add(Bidirectional(LSTM(12, return_sequences=False,activation="relu"), input_shape=(121,7)))
lstmModel2.add(Dropout(0.2))
lstmModel2.add(Dense(50, activation="relu"))
lstmModel2.add(Dense(1))
lstmModel2.compile(loss="mae",optimizer="adam")

In [ ]:
lstmModel2.summary()

In [ ]:
lstmScores2 = lstmModel2.fit(x_train,y_train,validation_split=0.3, epochs=20, shuffle=True,batch_size=50)

In [ ]:
lstmTargets = lstmModel2.predict(x_train)

In [ ]:
fig = pyplot.figure()
fig.suptitle('LSTM PREDICTIONS', fontsize=14)


pyplot.plot(y_train,label="actual targets", color="blue")
pyplot.plot(lstmTargets,label="predicted targets", color="red")


pyplot.xlabel('sample')
pyplot.ylabel('target')

pyplot.legend()
pyplot.show()

## GRU

In [ ]:
gruModel = Sequential()
gruModel.add(GRU(10,input_shape=(121,7), activation ="relu", return_sequences=False))
gruModel.add(Dropout(0.3,seed=seed_value))
gruModel.add(Dense(255, activation="relu"))
gruModel.add(Dense(1))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
gruModel.compile(loss="mae",optimizer=opt)

In [ ]:
gruModel.summary()

In [ ]:
gruScores = gruModel.fit(x_train,y_train,validation_split=0.3, epochs=50, shuffle=True)

In [ ]:
gruPredictions = gruModel.predict(x_train)

In [ ]:
fig = pyplot.figure()
fig.suptitle('DENSE CNN PREDICTIONS', fontsize=14)


pyplot.plot(y_train,label="actual targets", color="blue")
pyplot.plot(gruPredictions,label="predicted targets", color="red")


pyplot.xlabel('sample')
pyplot.ylabel('target')

pyplot.legend()
pyplot.show()

## k-fold CNN

In [34]:
cnn_fold_no = 1
cnnScores= []

In [35]:

for train, test in kfold.split(x_train, y_train):
    print("Fold:",cnn_fold_no)
    KCnnModel = Sequential()
    KCnnModel.add(Conv1D(filters=20, kernel_size=3, activation='relu', input_shape=(121, 7)))
    KCnnModel.add(MaxPooling1D(pool_size=2))
    KCnnModel.add(Flatten())
    KCnnModel.add(Dense(200, activation='relu'))
    KCnnModel.add(Dense(1))
    KCnnModel.compile(optimizer='adam', loss='mae')
    KFoldModelScores = KCnnModel.fit(x_train[train],y_train[train],epochs=10,batch_size=50,verbose=1)
    
    # Generate generalization metrics
    scores = KCnnModel.evaluate(x_train[test], y_train[test], verbose=0)
    print(scores)
    
    cnnScores.append(scores)
    
  
    
    cnn_fold_no = cnn_fold_no + 1
    

Train on 13984 samples
Epoch 1/10
13984/13984 [==============================] - 1s 77us/sample - loss: 30.6015
Epoch 2/10
13984/13984 [==============================] - 1s 55us/sample - loss: 27.1677
Epoch 3/10
13984/13984 [==============================] - 1s 55us/sample - loss: 26.8500
Epoch 4/10
13984/13984 [==============================] - 1s 55us/sample - loss: 26.5057
Epoch 5/10
13984/13984 [==============================] - 1s 55us/sample - loss: 26.2935
Epoch 6/10
13984/13984 [==============================] - 1s 55us/sample - loss: 26.0267
Epoch 7/10
13984/13984 [==============================] - 1s 55us/sample - loss: 25.5638
Epoch 8/10
13984/13984 [==============================] - 1s 57us/sample - loss: 25.3076
Epoch 9/10
 8350/13984 [================>.............] - ETA: 0s - loss: 24.8022

KeyboardInterrupt: 

In [33]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cnnScores), np.std(cnnScores)))

27.31% (+/- 1.12%)


## CNN

In [ ]:
cnnModel = Sequential()
cnnModel.add(Conv1D(filters=20, kernel_size=3, activation='relu', input_shape=(121, 7)))
cnnModel.add(MaxPooling1D(pool_size=2))
cnnModel.add(Flatten())
cnnModel.add(Dense(200, activation='relu'))
cnnModel.add(Dense(1))
cnnModel.compile(optimizer='adam', loss='mae')

In [ ]:
cnnModel.summary()

In [ ]:
cnnScores2 = cnnModel.fit(x_train, y_train, epochs=50, verbose=1,validation_split=0.3,shuffle=True)

In [ ]:
cnnModel2 = Sequential()
cnnModel2.add(Conv1D(filters=6, kernel_size=3, activation='relu', input_shape=(121, 7)))
cnnModel2.add(Conv1D(filters=12, kernel_size=3, activation='relu', input_shape=(121, 7)))
cnnModel2.add(MaxPooling1D(pool_size=2))
cnnModel2.add(Dropout(0.25))
cnnModel2.add(Flatten())
cnnModel2.add(Dense(128, activation='relu'))
cnnModel2.add(Dropout(0.25))
cnnModel2.add(Dense(1))
cnnModel2.compile(optimizer='adam', loss='mae')

In [ ]:
cnnModel2.summary()

In [ ]:
cnnScores3 = cnnModel2.fit(x_train, y_train, epochs=50, verbose=1,validation_split=0.3,shuffle=True)

In [ ]:
cnnPredicitions = cnnModel2.predict(x_train)

In [ ]:
cnnPredicitions = cnnPredicitions.flatten()

In [ ]:
cnnPredicitions.shape

In [ ]:
y_train.shape

In [ ]:
fig = pyplot.figure()
fig.suptitle('DENSE CNN PREDICTIONS', fontsize=14)


pyplot.plot(y_train,label="actual targets", color="blue")
pyplot.plot(cnnPredicitions,label="predicted targets", color="red")


pyplot.xlabel('sample')
pyplot.ylabel('target')

pyplot.legend()
pyplot.show()

## MULTI-HEAD CNN

In [ ]:
inputOne = Input(shape=(121,1))
cnn1 = Conv1D(filters=6, kernel_size=3, activation='relu')(inputOne)
cnn1 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputOne)
cnn1 = MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Dropout(0.25)(cnn1)
cnn1 = Flatten()(cnn1)

inputTwo = Input(shape=(121,1))
cnn2 = Conv1D(filters=6, kernel_size=3, activation='relu')(inputTwo)
cnn2 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputTwo)
cnn2 = MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Dropout(0.25)(cnn2)
cnn2 = Flatten()(cnn2)

inputThree = Input(shape=(121,1))
cnn3 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputThree)
cnn3 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputThree)
cnn3 = MaxPooling1D(pool_size=2)(cnn3)
cnn3 = Dropout(0.25)(cnn3)
cnn3 = Flatten()(cnn3)

inputFour = Input(shape=(121,1))
cnn4 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputFour)
cnn4 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputFour)
cnn4 = MaxPooling1D(pool_size=2)(cnn4)
cnn4 = Dropout(0.25)(cnn4)
cnn4 = Flatten()(cnn4)

inputFive = Input(shape=(121,1))
cnn5 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputFive)
cnn5 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputFive)
cnn5 = MaxPooling1D(pool_size=2)(cnn5)
cnn5 = Dropout(0.25)(cnn5)
cnn5 = Flatten()(cnn5)

inputSix = Input(shape=(121,1))
cnn6 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputSix)
cnn6 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputSix)
cnn6 = MaxPooling1D(pool_size=2)(cnn6)
cnn6 = Dropout(0.25)(cnn6)
cnn6 = Flatten()(cnn6)

inputSeven = Input(shape=(121,1))
cnn7 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputSeven)
cnn7 = Conv1D(filters=12, kernel_size=3, activation='relu')(inputSeven)
cnn7 = MaxPooling1D(pool_size=2)(cnn7)
cnn7 = Dropout(0.25)(cnn7)
cnn7 = Flatten()(cnn7)

merge = Concatenate()([cnn1, cnn2,cnn3,cnn4,cnn5,cnn6,cnn7])

dense = Dense(200, activation='relu')(merge)
dense2 = Dense(200, activation='relu')(dense)

output = Dense(1)(dense2)

MutliCnnModel = Model(inputs=[inputOne,inputTwo,inputThree,inputFour,inputFive,inputSix,inputSeven],outputs=output)

MutliCnnModel.compile(optimizer='adam',loss='mae')

In [ ]:
MutliCnnModel.summary()

In [ ]:
multiHeadCNNSCORESDEEPER = MutliCnnModel.fit([features[0],features[1],features[2],features[3],features[4],features[5],features[6]],y_train_cnn,epochs=50,validation_split=0.3,verbose=1)

In [ ]:
multiHeadTargets = MutliCnnModel.predict([features[0],features[1],features[2],features[3],features[4],features[5],features[6]])

In [ ]:
fig = pyplot.figure()
fig.suptitle('MULTI HEAD CNN TARGETS', fontsize=14)


pyplot.plot(y_train,label="actual targets", color="blue")
pyplot.plot(multiHeadTargets,label="predicted targets", color="red")


pyplot.xlabel('sample')
pyplot.ylabel('target')

pyplot.legend()
pyplot.show()

In [ ]:
fig = pyplot.figure()
fig.suptitle('LSTM VS DENSE CNN', fontsize=14)

# pyplot.plot(gruScores.history['loss'],label="GRU Train", color="red")
# pyplot.plot(gruScores.history['val_loss'],label="GRU Test", color="green")

# pyplot.plot(lstmScores.history['root_mean_squared_error'],label="LSTM Train", color="blue")
# pyplot.plot(lstmScores.history['val_root_mean_squared_error'],label="LSTM Test", color="brown")

pyplot.plot(lstmTanhScores.history['loss'],label="LSTM Train", color="red")
pyplot.plot(lstmTanhScores.history['val_loss'],label="LSTM Test", color="green")

pyplot.plot(lstmScores2.history['loss'],label="LSTM Train", color="blue")
pyplot.plot(lstmScores2.history['val_loss'],label="LSTM Test", color="black")

# pyplot.plot(cnnScores2.history['loss'],label="CNN Train", color="red")
# pyplot.plot(cnnScores2.history['val_loss'],label="CNN Test", color="green")

# pyplot.plot(cnnScores3.history['loss'],label="DENSE CNN Train", color="blue")
# pyplot.plot(cnnScores3.history['val_loss'],label="DENSE CNN Test", color="black")

# pyplot.plot(cnnScores2.history['loss'],label="CNN Train", color="blue")
# pyplot.plot(cnnScores2.history['val_loss'],label="CNN Test", color="black")


# pyplot.plot(multiHeadCNNSCORES.history['loss'],label="MULTI HEAD CNN Train", color="brown")
# pyplot.plot(multiHeadCNNSCORES.history['val_loss'],label="MULTI HEAD CNN Test", color="purple")


# pyplot.plot(multiHeadCNNSCORESDEEP.history['loss'],label="MULTI HEAD CNN 1 Train", color="red")
# pyplot.plot(multiHeadCNNSCORESDEEP.history['val_loss'],label="MULTI HEAD CNN 1 Test", color="green")

# pyplot.plot(multiHeadCNNSCORESDEEPER.history['loss'],label="MULTI HEAD CNN DEEP 1 Train", color="brown")
# pyplot.plot(multiHeadCNNSCORESDEEPER.history['val_loss'],label="MULTI HEAD CNN DEEP 1 Test", color="purple")

pyplot.xlabel('epoch')
pyplot.ylabel('loss(MAE)')

pyplot.legend()
pyplot.show()

## SCORES

In [ ]:
# LSTM SCORES
print("Lstm means:")
print(np.mean(lstmScores.history['loss']))
print(np.mean(lstmScores.history['val_loss']))
print(np.mean(lstmScores.history['root_mean_squared_error']))
print(np.mean(lstmScores.history['val_root_mean_squared_error']))
print("Lstm stds:")
print(np.std(lstmScores.history['loss']))
print(np.std(lstmScores.history['val_loss']))
print(np.std(lstmScores.history['root_mean_squared_error']))
print(np.std(lstmScores.history['val_root_mean_squared_error']))
print("")
# CNN SCORES
print("CNN means:")
print(np.mean(cnnScores.history['loss']))
print(np.mean(cnnScores.history['val_loss']))
print(np.mean(cnnScores.history['root_mean_squared_error']))
print(np.mean(cnnScores.history['val_root_mean_squared_error']))
print("CNN stds:")
print(np.std(cnnScores.history['loss']))
print(np.std(cnnScores.history['val_loss']))
print(np.std(cnnScores.history['root_mean_squared_error']))
print(np.std(cnnScores.history['val_root_mean_squared_error']))
print("")
# CNN2 SCORES
print("CNN2 means:")
print(np.mean(cnnScores3.history['loss']))
print(np.mean(cnnScores3.history['val_loss']))
print(np.mean(cnnScores3.history['root_mean_squared_error']))
print(np.mean(cnnScores3.history['val_root_mean_squared_error']))
print("CNN2 stds:")
print(np.std(cnnScores3.history['loss']))
print(np.std(cnnScores3.history['val_loss']))
print(np.std(cnnScores3.history['root_mean_squared_error']))
print(np.std(cnnScores3.history['val_root_mean_squared_error']))
print("")
# MUTLI HEAD CNN SCORES
print("MUTLI HEAD CNN means:")
print(np.mean(multiHeadCNNSCORES.history['loss']))
print(np.mean(multiHeadCNNSCORES.history['val_loss']))
# print(np.mean(multiHeadCNNSCORES.history['root_mean_squared_error']))
# print(np.mean(multiHeadCNNSCORES.history['val_root_mean_squared_error']))
print("MUTLI HEAD CNN stds:")
print(np.std(multiHeadCNNSCORES.history['loss']))
print(np.std(multiHeadCNNSCORES.history['val_loss']))
# print(np.std(multiHeadCNNSCORES.history['root_mean_squared_error']))
# print(np.std(multiHeadCNNSCORES.history['val_root_mean_squared_error']))
print("")
#GRU SCORES
print("GRU means:")
print(np.mean(gruScores.history['loss']))
print(np.mean(gruScores.history['val_loss']))
print(np.mean(gruScores.history['root_mean_squared_error']))
print(np.mean(gruScores.history['val_root_mean_squared_error']))
print("GRU stds:")
print(np.std(gruScores.history['loss']))
print(np.std(gruScores.history['val_loss']))
print(np.std(gruScores.history['root_mean_squared_error']))
print(np.std(gruScores.history['val_root_mean_squared_error']))


In [ ]:
trainPredict = model.predict(x_train)

In [ ]:
predictedLabels = model.predict(x_test)

In [ ]:
predictedLabels = predictedLabels.flatten()